In [ ]:
import os
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple
import pandas as pd
import numpy as np
from pathlib import Path
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from langchain_openai import ChatOpenAI
import yfinance as yf
from dataclasses import dataclass
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv()

# Validate environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OpenAI API key not found in environment variables")


@dataclass
class AnalysisConfig:
    """Configuration for analysis parameters"""

    period_years: int = 5
    interval: str = "1wk"  # 1d, 1wk, 1mo
    moving_averages: List[int] = None
    rsi_period: int = 14
    volume_ma_period: int = 20

    def __post_init__(self):
        if self.moving_averages is None:
            self.moving_averages = [50, 200]  # Default MAs


class Tasks:
    def research(self, agent):
        return Task(
            description="""Research the company with ticker {company} over the past 5 years. 
            1. Analyze major news events and their impact
            2. Track management changes and strategic shifts
            3. Evaluate competitive position changes
            4. Assess market sentiment trends
            5. Review analyst coverage and ratings history
            The company ticker is a required input parameter that will be provided.""",
            agent=agent,
            expected_output="""A comprehensive research report including:
                1. Timeline of major events and their market impact
                2. Changes in company strategy and execution
                3. Market sentiment analysis with specific examples
                4. Competitive position assessment
                5. Notable analyst opinions and rating changes""",
        )

    def technical_analysis(self, agent):
        return Task(
            description="""Perform detailed technical analysis for ticker {company} with 5-year historical data.
            1. Identify primary and secondary trends with SPECIFIC PRICE LEVELS
            2. Calculate EXACT support/resistance levels in numbers
            3. Analyze volume patterns with SPECIFIC percentage changes
            4. Calculate and interpret RSI, MACD values
            5. Compare current price to 50, 100, 200-day MAs
            6. Provide EXACT price targets
            YOU MUST PROVIDE SPECIFIC NUMBERS AND EXACT VALUES.""",
            agent=agent,
            expected_output="""A technical analysis report with EXACT numbers including:
                1. Current price: $XX.XX
                2. Primary trend: UP/DOWN with XX% change over past Y months
                3. Support levels: $XX.XX, $XX.XX
                4. Resistance levels: $XX.XX, $XX.XX
                5. RSI: XX.XX (OVERBOUGHT/NEUTRAL/OVERSOLD)
                6. Volume: XX% above/below 20-day average
                7. Price targets: 
                - Upside: $XX.XX (XX% potential)
                - Downside: $XX.XX (XX% risk)
                8. Clear BULLISH/BEARISH/NEUTRAL rating based on technicals""",
        )

    def financial_analysis(self, agent):
        return Task(
            description="""Conduct comprehensive financial analysis for company ticker {company}.
            1. Analyze 5-year trends in key metrics
            2. Evaluate capital structure and efficiency
            3. Assess profitability and growth metrics
            4. Review cash flow generation and usage
            5. Compare with industry benchmarks
            6. Examine insider transactions and ownership changes
            The company ticker is a required input parameter that will be provided.""",
            agent=agent,
            expected_output="""A detailed financial analysis including:
                1. Growth trend analysis with specific metrics
                2. Profitability analysis with industry comparisons
                3. Balance sheet strength assessment
                4. Cash flow analysis and capital allocation
                5. Key risk factors identification
                6. Insider activity patterns and implications""",
        )

    def investment_recommendation(self, agent, context_tasks):
        return Task(
            description="""Synthesize all analyses to provide a detailed investment recommendation for {company}.
            1. Evaluate all technical and fundamental factors
            2. Assess risk-reward ratio
            3. Consider market conditions and timing
            4. Provide specific price targets
            5. Identify key catalysts and risks
            The company ticker is a required input parameter that will be provided.""",
            agent=agent,
            context_tasks=context_tasks,
            expected_output="""A specific investment recommendation including:
                1. Clear STRONG BUY / BUY / HOLD / SELL / STRONG SELL rating
                2. Specific price targets (entry, exit, stop-loss)
                3. Expected timeframe for the investment
                4. Key catalysts to monitor
                5. Risk factors and mitigation strategies
                6. Position sizing recommendations""",
        )


def save_analysis_to_markdown(result: str, ticker: str) -> Path:
    """Save the analysis result to a markdown file"""
    reports_dir = Path("reports")
    reports_dir.mkdir(exist_ok=True)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = reports_dir / f"{ticker}_analysis_{timestamp}.md"

    content = f"""# Investment Analysis Report for {ticker}
Generated on: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

## Analysis Summary
{result}

---
*This report was generated automatically by the Stock Analysis System*
"""

    with open(filename, "w", encoding="utf-8") as f:
        f.write(content)

    return filename


class FinancialMetrics:
    """Calculate financial metrics and ratios"""

    @staticmethod
    def calculate_ratios(financial_data: pd.DataFrame) -> Dict:
        try:
            current_ratio = (
                financial_data["Total Current Assets"]
                / financial_data["Total Current Liabilities"]
            )
            debt_to_equity = (
                financial_data["Total Liabilities"]
                / financial_data["Total Stockholder Equity"]
            )
            return {
                "current_ratio": current_ratio.iloc[-1],
                "debt_to_equity": debt_to_equity.iloc[-1],
                "quick_ratio": (
                    financial_data["Total Current Assets"] - financial_data["Inventory"]
                ).iloc[-1]
                / financial_data["Total Current Liabilities"].iloc[-1],
                "roe": (
                    financial_data["Net Income"]
                    / financial_data["Total Stockholder Equity"]
                ).iloc[-1],
            }
        except Exception as e:
            logger.error(f"Error calculating financial ratios: {e}")
            return {}


class TechnicalAnalysis:
    """Technical analysis calculations"""

    @staticmethod
    def calculate_rsi(data: pd.DataFrame, period: int = 14) -> pd.Series:
        delta = data["Close"].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))

    @staticmethod
    def calculate_support_resistance(
        data: pd.DataFrame, window: int = 20
    ) -> Tuple[float, float]:
        rolling_min = data["Low"].rolling(window=window).min()
        rolling_max = data["High"].rolling(window=window).max()
        return rolling_min.iloc[-1], rolling_max.iloc[-1]


class ComprehensivePriceTool(BaseTool):
    name: str = "Comprehensive Price Analysis"
    description: str = "Get detailed historical price data with technical indicators"

    def __init__(self, config: AnalysisConfig):
        super().__init__()
        self._config = config

    def _run(self, ticker: str) -> str:
        try:
            stock = yf.Ticker(ticker)
            history = stock.history(
                period=f"{self._config.period_years}y", interval=self._config.interval
            )

            if history.empty:
                return "No price data available for this ticker"

            # Calculate technical indicators
            for ma in self._config.moving_averages:
                history[f"MA_{ma}"] = history["Close"].rolling(window=ma).mean()

            history["RSI"] = TechnicalAnalysis.calculate_rsi(
                history, self._config.rsi_period
            )
            history["Volume_MA"] = (
                history["Volume"].rolling(window=self._config.volume_ma_period).mean()
            )

            support, resistance = TechnicalAnalysis.calculate_support_resistance(
                history
            )

            analysis_summary = {
                "price_data": history.to_csv(),
                "current_price": history["Close"].iloc[-1],
                "support_level": support,
                "resistance_level": resistance,
                "rsi": history["RSI"].iloc[-1],
                "volume_trend": (
                    "Up"
                    if history["Volume"].iloc[-1] > history["Volume_MA"].iloc[-1]
                    else "Down"
                ),
            }

            return str(analysis_summary)
        except Exception as e:
            logger.error(f"Error in price analysis: {e}")
            return f"Error analyzing price data: {str(e)}"


class FinancialMetricsTool(BaseTool):
    name: str = "Financial Metrics Analysis"
    description: str = "Analyze comprehensive financial metrics and growth trends"

    def _run(self, ticker: str) -> str:
        try:
            stock = yf.Ticker(ticker)

            # Get various financial statements
            balance_sheet = stock.balance_sheet
            income_stmt = stock.income_stmt
            cash_flow = stock.cashflow

            # Calculate key metrics and growth rates
            metrics = {
                "balance_sheet": balance_sheet.to_csv(),
                "income_statement": income_stmt.to_csv(),
                "cash_flow": cash_flow.to_csv(),
                "key_ratios": FinancialMetrics.calculate_ratios(balance_sheet),
                "revenue_growth": income_stmt.loc["Total Revenue"].pct_change().mean(),
                "profit_margin": (
                    income_stmt.loc["Net Income"] / income_stmt.loc["Total Revenue"]
                ).mean(),
            }

            return str(metrics)
        except Exception as e:
            logger.error(f"Error in financial analysis: {e}")
            return f"Error analyzing financial data: {str(e)}"


class AdvancedAgents:
    def __init__(self, config: AnalysisConfig):
        self.config = config

    def financial_analyst(self) -> Agent:
        return Agent(
            role="Senior Financial Analyst",
            goal="Provide comprehensive financial analysis focusing on long-term trends and company health",
            backstory="""A veteran financial analyst with 20 years of experience in equity research. 
                     Specialized in identifying long-term value opportunities through deep fundamental analysis.""",
            tools=[FinancialMetricsTool()],
            verbose=True,
        )

    def technical_analyst(self) -> Agent:
        return Agent(
            role="Technical Analysis Expert",
            goal="Analyze long-term price patterns and technical indicators to identify major trends",
            backstory="""A technical analysis expert with expertise in pattern recognition and trend analysis. 
                     Known for identifying major market turning points using multi-timeframe analysis.""",
            tools=[ComprehensivePriceTool(self.config)],
            verbose=True,
        )

    def hedge_fund_manager(self) -> Agent:
        return Agent(
            role="Hedge Fund Manager",
            goal="Synthesize all analyses to provide a clear investment recommendation",
            backstory="""A seasoned hedge fund manager with 25 years of experience in global markets.
              Knwon for making decisive investment calls on whether to 'Sell', 'Buy', or 'Hold' based on comprehensive analysis """,
            tools=[],  # Uses ananlysis from other agents
            verbose=True,
        )


def create_analysis_crew(
    ticker: str, config: AnalysisConfig = AnalysisConfig()
) -> Tuple[str, Path]:
    try:
        # Initialize agents and tasks
        agents = AdvancedAgents(config)
        tasks = Tasks()

        # Create analysts
        financial_analyst = agents.financial_analyst()
        technical_analyst = agents.technical_analyst()
        hedge_fund_manager = agents.hedge_fund_manager()

        # Define tasks
        financial_task = tasks.financial_analysis(financial_analyst)
        technical_task = tasks.technical_analysis(technical_analyst)

        # Final recommendation task uses results from previous analyses
        recommend_task = tasks.investment_recommendation(
            hedge_fund_manager,
            context_tasks=[financial_task, technical_task],
        )

        # Create and configure crew
        crew = Crew(
            agents=[financial_analyst, technical_analyst, hedge_fund_manager],
            tasks=[financial_task, technical_task, recommend_task],
            verbose=True,
            process=Process.sequential,
            manager_llm=ChatOpenAI(
                model_name="gpt-4o-mini",
                temperature=0.5,
                api_key=OPENAI_API_KEY,
            ),
        )

        # Execute analysis
        result = crew.kickoff(inputs={"company": ticker})

        # Save the result to markdown
        report_file = save_analysis_to_markdown(result, ticker)

        return result, report_file

    except Exception as e:
        logger.error(f"Error in analysis execution: {e}")
        raise


if __name__ == "__main__":
    try:
        # Configure analysis parameters
        config = AnalysisConfig(
            period_years=5,
            interval="1wk",
            moving_averages=[50, 100, 200],
            rsi_period=14,
            volume_ma_period=20,
        )

        # Run analysis
        result, report_file = create_analysis_crew("066570.KS", config)
        print(f"Analysis completed. Report saved to: {report_file}")
        print("\nAnalysis Result:")
        print(result)

    except Exception as e:
        logger.error(f"Analysis failed: {e}")